<a href="https://colab.research.google.com/github/jyothi4218/gluacoma_detection_mobilenetv2/blob/main/gluacomadetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import kagglehub
import os

# 1️⃣ Download Dataset
dataset_folder = kagglehub.dataset_download("turpatieswarprasad/gluacoma-dataset")

# 2️⃣ Check Folder Structure
print("Dataset directory:", dataset_folder)
print("Contents:", os.listdir(dataset_folder))

# 3️⃣ Adjust Paths
if "train" in os.listdir(dataset_folder):  # If train folder exists
    train_dir = os.path.join(dataset_folder, "train")
    val_dir = os.path.join(dataset_folder, "val")
else:  # If no subfolders, use main folder
    train_dir = dataset_folder
    val_dir = dataset_folder

# 4️⃣ Load Dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary"
)

val_data = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="binary"
)


Dataset directory: /kaggle/input/gluacoma-dataset
Contents: ['glaucoma_dataset']
Found 17234 images belonging to 1 classes.
Found 17234 images belonging to 1 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load Pretrained MobileNetV2 Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze the base model

# Build Custom Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (Glaucoma vs. Normal)
])

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Train the Model
EPOCHS = 10
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)

# Evaluate on Validation Data
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Save the Model
model.save("glaucoma_detection_model.h5")
print("Model saved successfully!")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
539/539 ━━━━━━━━━━━━━━━━━━━━ 414s 748ms/step - accuracy: 0.9762 - loss: 0.0577 - val_accuracy: 1.0000 - val_loss: 1.2284e-05
Epoch 2/10
539/539 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 1.0000 - loss: 2.5918e-05

KeyboardInterrupt: 